## This code performs whole-brain simulations where thermo- and hygrosensory neurons are activated and the activity of all other neurons is recorded.

### Simulations are based on the leaky integrate and fire model by Shiu et al. (bioRxiv, 2023). 

### This notebook is adapted from one generously shared by Philip Shiu et al. (https://github.com/philshiu/Drosophila_brain_model)

In [24]:
from model import run_exp
from model import default_params as params
import utils as utl
from brian2 import Hz

config = {
    'path_res'  : './results/connectome_paper',                     # directory to store results
    'path_comp' : './ShiuFiles/Completeness_783.csv',        # csv of the complete list of Flywire neurons
    'path_con'  : './ShiuFiles/Connectivity_783.parquet',    # connectivity data
    'n_proc'    : -1,                                               # number of CPU cores (-1: use all)
}

## Underlying connectivity data
The connectivity of the fly brain is stored in the folowing files:
- neurons present: `config['path_comp']`
- connectivity between neurons: `config['path_con]`

## Model parameters
The equation and constants for the leaky integrate and fire model are defined 
in the dictionary `default_params` in the beginning of the file `model.py`:

```
default_params = {
    # trials
    't_run'     : 1000 * ms,              # duration of trial
    'n_run'     : 30,                     # number of runs

    'v_0'       : -52 * mV,               # resting potential
    'v_rst'     : -52 * mV,               # reset potential after spike
    [...]
```
We can also change values
and pass the modified dictionary to the model (see Experiment 1).

## First, define each set of GRNs.
## We are using the same neurons as for the connectome analyses.

In [25]:
# Thermosensory neurons in left hemisphere
thermo_L = [
    720575940615856345,
    720575940613794114,
    720575940646372228,
    720575940619024774,
    720575940616039197,
    720575940625293514,
    720575940623793741,
    720575940627944464,
    720575940618512624,
    720575940622972820,
    720575940638389437,
    720575940616999069,
    720575940613191591
    ]

# Thermosensory neurons in right hemisphere
thermo_R = [
    720575940619930534,
    720575940611720362,
    720575940617041728,
    720575940631875298,
    720575940646785924,
    720575940621659563,
    720575940627135306,
    720575940627677258,
    720575940636878254,
    720575940635969399,
    720575940632913461,
    720575940633059381,
    720575940621831804,
    720575940629007230,
    720575940650751873,
    720575940622082518
    ]

# Hygrosensory neurons in left hemisphere
hygro_L = [
    720575940638509952,
    720575940616239873,
    720575940624673936,
    720575940614634786,
    720575940617969961,
    720575940620634537,
    720575940630644281,
    720575940635767524,
    720575940633280107,
    720575940618574983,
    720575940624011278,
    720575940627150991,
    720575940635916942,
    720575940622581651,
    720575940614921501,
    720575940617665950,
    720575940614570793,
    720575940617986089,
    720575940603820716,
    720575940617986345,
    720575940617987113,
    720575940605264305,
    720575940614996018,
    720575940623065013,
    720575940620459702,
    720575940638300093,
    720575940622457028,
    720575940622700749,
    720575940609792206,
    720575940614711638,
    720575940618531544,
    720575940637096026,
    720575940610460514,
    720575940628945516,
    720575940651079158,
    720575940626580361,
    720575940604931249,
    720575940622625097,
    720575940615553397
    ]

# Hygrosensory neurons in right hemisphere
hygro_R = [
    720575940613914774,
    720575940620602236,
    720575940619075073,
    720575940614842262,
    720575940625868184,
    720575940633274524,
    720575940614919709,
    720575940623476661,
    720575940637494837,
    720575940630282297,
    720575940626125626,
    720575940627271230,
    720575940623030861,
    720575940620644446,
    720575940626256992,
    720575940632714465,
    720575940611377893,
    720575940620847086,
    720575940643707246,
    720575940629693302,
    720575940630271862,
    720575940648747385,
    720575940621009023,
    720575940627264262,
    720575940625148174,
    720575940613942045,
    720575940613941533,
    720575940613942301,
    720575940613942557,
    720575940634781620,
    720575940625657928,
    720575940622112726,
    720575940623209703,
    720575940633670904,
    720575940629275898
    ]

In [26]:
flyid2name_thermoL = { f: 'sugar_L_{}'.format(i+1) for i, f in enumerate(thermo_L) }
flyid2name_hygroL = { f: 'bitter_L_{}'.format(i+1) for i, f in enumerate(hygro_L) }

# view example
flyid2name_hygroL

{720575940638509952: 'bitter_L_1',
 720575940616239873: 'bitter_L_2',
 720575940624673936: 'bitter_L_3',
 720575940614634786: 'bitter_L_4',
 720575940617969961: 'bitter_L_5',
 720575940620634537: 'bitter_L_6',
 720575940630644281: 'bitter_L_7',
 720575940635767524: 'bitter_L_8',
 720575940633280107: 'bitter_L_9',
 720575940618574983: 'bitter_L_10',
 720575940624011278: 'bitter_L_11',
 720575940627150991: 'bitter_L_12',
 720575940635916942: 'bitter_L_13',
 720575940622581651: 'bitter_L_14',
 720575940614921501: 'bitter_L_15',
 720575940617665950: 'bitter_L_16',
 720575940614570793: 'bitter_L_17',
 720575940617986089: 'bitter_L_18',
 720575940603820716: 'bitter_L_19',
 720575940617986345: 'bitter_L_20',
 720575940617987113: 'bitter_L_21',
 720575940605264305: 'bitter_L_22',
 720575940614996018: 'bitter_L_23',
 720575940623065013: 'bitter_L_24',
 720575940620459702: 'bitter_L_25',
 720575940638300093: 'bitter_L_26',
 720575940622457028: 'bitter_L_27',
 720575940622700749: 'bitter_L_28',
 

# Running simulations
## Background info:
To run a simulation exciting these nerons we have to call `run_exp` supplying the following:
- unique name for the simulation: `exp_name`
- a list of neurons we want to stimulate: `thermo_L`
- the connectivity data: `config['path_comp']` and `config['path_con]`
- path to store the output: `config['path_res']`
- number of CPU cores use: `config['n_procs]`

The `.parquet` file created during a simulation contains all spikes events of all neurons in the model.
We load the data again from disk by passing a list of result files to the `utl.load_exps` function.

The spike times can be converted to spike rates [Hz] via `utl.get_rate`, which requires the duration of each trial.
`utl.get_rate` returns `pandas.DataFrame` objects:
1. spike rate for each neuron (rows) in each experiment (column): `df_rate`
2. standard deviation of rate across trials: `df_rate_std`

For convenience, we can optionally pass the `flyid2name` dictionary to `utl.get_rate` in order to convert flywire IDs into
meaningful names.

In [27]:
#show default params
params

{'t_run': 1. * second,
 'n_run': 30,
 'v_0': -52. * mvolt,
 'v_rst': -52. * mvolt,
 'v_th': -45. * mvolt,
 't_mbr': 20. * msecond,
 'tau': 5. * msecond,
 't_rfc': 2.2 * msecond,
 't_dly': 1.8 * msecond,
 'w_syn': 275. * uvolt,
 'r_poi': 25. * hertz,
 'r_poi2': 0. * hertz,
 'f_poi': 250,
 'eqs': '\ndv/dt = (v_0 - v + g) / t_mbr : volt (unless refractory)\ndg/dt = -g / tau               : volt (unless refractory) \nrfc                            : second\n',
 'eq_th': 'v > v_th',
 'eq_rst': 'v = v_rst; w = 0; g = 0 * mV'}

## Thermosensory neuron activation

In [28]:
# Run simulation at diff stim intensities

for stim_rate in [25,50,75,100,125,150,175,200]: 
    
    prefix = 'thermo_' + str(stim_rate) + 'Hz'
    params['r_poi'] = stim_rate * Hz
    run_exp(exp_name=prefix, neu_exc=thermo_L, params=params, **config)
    
    # extract results
    datafilename = './results/connectome_paper/' + prefix + '.parquet'
    df_spike = utl.load_exps([datafilename])
    df_rate, df_rate_std = utl.get_rate(df_spike, t_run=params['t_run'], n_run=params['n_run'], flyid2name=flyid2name_thermoL)
    
    # save dataframes to csv
    savepath = 'results/connectome_paper/'
    df_rate.fillna(0).to_csv(savepath + prefix + '_rates.csv')
    df_rate_std.fillna(0).to_csv(savepath + prefix + '_std.csv')

>>> Experiment:     thermo_25Hz
    Output file:    results/connectome_paper/thermo_25Hz.parquet
    Excited neurons: 13


KeyboardInterrupt: 